In [1]:
import os
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt

from utils import constants

Load pre-processed data file and isolate phase data

In [2]:
data_dir = constants.PHASE_DATA_DIR

all_data = pd.read_parquet(os.path.join(constants.ALL_SIGNAL_DIR, 'all_filt_chan.parquet'))

In [3]:
# Filter phase features only
ph_data = all_data[[c for c in all_data.columns if ('ac_' not in c) and ('dc_' not in c)]]

ph_data.to_parquet(os.path.join(data_dir, 'phase_filt_chan.parquet'), index=False)

We can use a specific window of the signal or the full signal

In [20]:
ph_data = pd.read_parquet(os.path.join(data_dir, 'phase_filt_chan.parquet'))

ph_data = ph_data.rename({'trial_num_04': 'trial_num',
                          'subject_id_04': 'subject_id',
                          'trial_type_04': 'trial_type',
                          'montage_04': 'montage'}, axis=1)
ph_data = ph_data.drop(['trial_num_08', 'subject_id_08', 'trial_type_08', 'montage_08',
                        'trial_num_13', 'subject_id_13', 'trial_type_13', 'montage_13'], axis=1)

In [21]:
# Utilize only the full signal
ph_all = ph_data[[c for c in ph_data.columns if c.split('_')[0] == 'ph'] + ['trial_num', 'subject_id', 'trial_type', 'montage']]

ph_all.to_parquet(os.path.join(data_dir, 'phase_all_filt_chan.parquet'), index=False)

In [6]:
# Filter RT window for phase data only
ph_RT = ph_data[[c for c in ph_data.columns if c.split('_')[0] == 'ph-rt'] + ['trial_num', 'subject_id', 'trial_type', 'montage']]

ph_RT.to_parquet(os.path.join(data_dir, 'phase_rt_filt_chan.parquet'), index=False)

In [7]:
# Filter pre-stim window for phase data only
ph_pre_stim = ph_data[[c for c in ph_data.columns if c.split('_')[0] == 'ph-pre-stim'] + ['trial_num', 'subject_id', 'trial_type', 'montage']]

ph_pre_stim.to_parquet(os.path.join(data_dir, 'phase_pre_stim_filt_chan.parquet'), index=False)

In [8]:
# Filter initial response window for phase data only
ph_init = ph_data[[c for c in ph_data.columns if c.split('_')[0] == 'ph-init'] + ['trial_num', 'subject_id', 'trial_type', 'montage']]

ph_init.to_parquet(os.path.join(data_dir, 'phase_init_filt_chan.parquet'), index=False)

In [9]:
# Filter pre-RT window for phase data only
ph_pre_RT = ph_data[[c for c in ph_data.columns if c.split('_')[0] == 'ph-pre-rt'] + ['trial_num', 'subject_id', 'trial_type', 'montage']]

ph_pre_RT.to_parquet(os.path.join(data_dir, 'phase_pre_rt_filt_chan.parquet'), index=False)

In [10]:
# Filter post-RT window for phase data only
ph_post_RT = ph_data[[c for c in ph_data.columns if c.split('_')[0] == 'ph-post-rt'] + ['trial_num', 'subject_id', 'trial_type', 'montage']]

ph_post_RT.to_parquet(os.path.join(data_dir, 'phase_post_rt_filt_chan.parquet'), index=False)

# Extract band power features

In [12]:
montages = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']

window_mapping = {
    'all': 'ph',
    'rt': 'ph-rt',
    'pre-stim': 'ph-pre-stim',
    'init': 'ph-init',
    'pre-rt': 'ph-pre-rt',
    'post-rt': 'ph-post-rt'
}

def extract_features(data_dir, input_fname, key, output_fname):

    df = pd.read_parquet(os.path.join(data_dir, input_fname))
    length = len(df.loc[0, f'{window_mapping[key]}_e_0_04'])
    orig = df[[f'{window_mapping[key]}_{m}_{i}_{f}' for m in montages for i in range(128) for f in ['04', '08', '13']]].values
    empty = np.zeros((orig.shape[0], orig.shape[1], length))

    for i in range(orig.shape[0]):
        for j in range(orig.shape[1]):
            empty[i, j, :] = orig[i, j]
    max_feats = pd.DataFrame(np.max(empty.copy(), axis=-1),
        columns=[f'max_{window_mapping[key]}_{m}_{i}_{f}' for m in montages for i in range(128) for f in ['04', '08', '13']])
    min_feats = pd.DataFrame(np.min(empty.copy(), axis=-1),
        columns=[f'min_{window_mapping[key]}_{m}_{i}_{f}' for m in montages for i in range(128) for f in ['04', '08', '13']])
    mean_feats = pd.DataFrame(np.mean(empty.copy(), axis=-1),
        columns=[f'mean_{window_mapping[key]}_{m}_{i}_{f}' for m in montages for i in range(128) for f in ['04', '08', '13']])
    range_feats = pd.DataFrame((np.max(empty.copy(), axis=-1) - np.min(empty.copy(), axis=-1)),
        columns=[f'range_{window_mapping[key]}_{m}_{i}_{f}' for m in montages for i in range(128) for f in ['04', '08', '13']])
    avg_pwr_feats = pd.DataFrame(np.sum(empty.copy() ** 2,  axis=-1) / empty.shape[2],
        columns=[f'avg_pwr_{window_mapping[key]}_{m}_{i}_{f}' for m in montages for i in range(128) for f in ['04', '08', '13']])

    nan_mask = np.isnan(empty.copy()).any(axis=-1)
    new_values = np.sum(empty.copy() > 0,  axis=-1)
    new_values = new_values.astype('float64')
    new_values[nan_mask] = np.nan

    samp_gt_zero_feats = pd.DataFrame(new_values,
        columns=[f'samp_gt_zero_{window_mapping[key]}_{m}_{i}_{f}' for m in montages for i in range(128) for f in ['04', '08', '13']])
    
    def num_zero_crossing(arr):
        return len(np.where(np.diff(np.signbit(arr)))[0])

    nan_mask = np.isnan(empty.copy()).any(axis=-1)
    new_values = np.apply_along_axis(num_zero_crossing, axis=-1, arr=empty.copy())
    new_values = new_values.astype('float64')
    new_values[nan_mask] = np.nan

    zero_cross_feats = pd.DataFrame(new_values,
        columns=[f'zero_cross_{window_mapping[key]}_{m}_{i}_{f}' for m in montages for i in range(128) for f in ['04', '08', '13']])
    
    feats_df = pd.concat([max_feats, min_feats, mean_feats, range_feats, avg_pwr_feats, samp_gt_zero_feats, zero_cross_feats], axis=1)
    info_df = df[['trial_num', 'subject_id', 'trial_type', 'montage']]
    final_df = pd.concat([info_df, feats_df], axis=1)
    final_df.dropna(axis=1, how='all', inplace=True)

    print('Writing output file...')
    final_df.to_parquet(os.path.join(data_dir, output_fname), index=False)

Without CSP

In [ ]:
data_dir = os.path.join(constants.PYTHON_DIR, 'phase_data')
key = 'all'
input_fname = f'phase_{key}_filt_chan.parquet'
output_fname = f'{key}_simple_bandpower_features.parquet'

extract_features(data_dir, input_fname, key, output_fname)

With CSP

In [ ]:
data_dir = os.path.join(constants.PYTHON_DIR, 'csp_transform')
key = 'all'
n_filters = 16
input_fname = f'CSP_filt_{n_filters}_{key}.parquet'
output_fname = f'simple_bandpower_features_csp_{n_filters}_{key}.parquet'

extract_features(data_dir, input_fname, key, output_fname)